### Functions

In [174]:
# choose execl 
def choose_file():
    
    #return 'FFG Holdings+Opt 05-11-18 NEW' + '.xlsx'
    return 'TDA 08-27-18' + '.xlsx'

In [94]:
# input shares dates
def get_date():
    
    # !IMPORTANT! add 1 day to period, because `get` returns previous day
    return datetime.date(2018, 8, 28) + timedelta(days=1)  
    #return datetime.datetime.now()


In [95]:
# read xls into pandas table
def create_table(file_name):
    tables_list = pd.read_excel( file_name, sheetname=None)
    
    # get last worksheet from list
    table = tables_list[list(tables_list.keys())[-1]]
    
    # clear nan rows
    table = table[pd.isnull(table.SYMBOL) == False]
    
    # extract existing found-codes
    symbl = table.SYMBOL.dropna().unique()
    
    return table, symbl  # return table and list of found market symbols

In [96]:
# read and preprocess shares from yahoo on current date
def read_shares(symbl, start, end):

    # set date interval
    start = end = get_date()

    # download into dataPanel
    shares_panel = web.get_data_yahoo(list(symbl), start, end)
 
    # convert Panel to the Multyindexed frame
    shares = shares_panel.to_frame()

    # remove index level 0 (date)
    shares.index = shares.index.droplevel(0)

    # convert index into column
    shares.reset_index(inplace=True)

    # rename `minor` column
    shares.rename(columns={'minor': 'SYMBOL'}, inplace=True)

    return shares

In [178]:
# create output xls from table and shares dataframes
def create_output(table, shares, start):
    
    # remove unnesessary columns from shares
    columns_to_drop = ['Close', 'High', 'Low', 'Open', 'Volume']
    shares.drop(columns_to_drop, axis=1, inplace=True)
    
    # create table
    table = pd.merge(left=table, right=shares, left_on='SYMBOL', right_on='SYMBOL')
    
     # reformat prices to 2 dec.digits
    table['Adj Close'] = table['Adj Close'].map(lambda x: round(x, 2))
    
    # rename some columns
    new_col_names = {'STOCKS':'STOCKS-BUY','Unnamed: 10':'STOCKS-SELL',
                'CALLS':'CALLS-BUY','Unnamed: 12': 'CALLS-SELL',
                'PUTTES':'PUTTES-BUY','Unnamed: 14': 'PUTTES-SELL'}
    table.rename(columns=new_col_names, inplace=True)
    
    # insert `CURRENT DATE` column before `ADJ price`
    table.insert(len(table.columns)-1,'CURRENT DATE', start)
    
    table['STRIKE UPSIDE'] = table['STRIKE UPSIDE %'] = None

    
    table.loc[table['TRX CODE'].str[0] == 'O', 'STRIKE UPSIDE'] = table['Adj Close'] - table['Strike  Price']
    table.loc[table['TRX CODE'].str[0] == 'E', 'STRIKE UPSIDE'] = table['Adj Close'] - table['BASIS COST']
    
    table.loc[table['TRX CODE'].str[0] == 'O', 'STRIKE UPSIDE %'] = table['STRIKE UPSIDE'] / table['Strike  Price']
    table.loc[table['TRX CODE'].str[0] == 'E', 'STRIKE UPSIDE %'] = table['STRIKE UPSIDE'] / table['BASIS COST']

    # write new xls table
    table.to_excel('result_' + file_name, index=False)
    
    return table


### Start here

In [98]:
# main part
import os
import pandas as pd
#import pandas_datareader.data as web
from pandas_datareader import data as web

import fix_yahoo_finance as yf
yf.pdr_override() # <== that's all it takes :-)

# download dataframe
#data = web.get_data_yahoo('AAPL', start="2018-05-10", end="2018-05-10")
from datetime import date
from datetime import datetime
import datetime
from datetime import timedelta

In [179]:
# works chain
file_name = choose_file()
start = end = get_date()
table, symbl = create_table(file_name)  # create pandas df
print ('\nwork tables created...')
shares = read_shares(symbl, start, end)  # deliver shares from yahoo
if shares.empty:
    print 'can not grab shares from yahoo-finance\ntry again in few minutes, please'
    exit
else:
    print ('shares delivered...')
    table = create_output(table, shares, start)  # create, reformat and write output xls
    print ('output file created')



work tables created...
[*********************100%***********************]  31 of 31 downloaded
shares delivered...
output file created


In [30]:
exit 1

### Sandbox space

In [116]:
shares.head(3)

,SYMBOL,Adj Close
0,AAPL,219.699997
1,AGN,187.720001
2,AMAT,43.400002


In [162]:
table.loc[table['TRX  CODE'].str[0] == 'E', 'Quantity'] = 0

In [163]:
table.iloc[0].SYMBOL=33333

In [171]:
table[:15]

,TRX Date,SYMBOL,Expir Date,TRX CODE,Quantity,BASIS COST,Strike Price,Call Premium,Market at TRX,STOCKS-BUY,...,PUTTES-BUY,PUTTES-SELL,CASH,DIVDNT,Gain/ (Loss),BALANCE,CURRENT DATE,Adj Close,STRIKE UPSIDE,STRIKE UPSIDE %
0,2018-06-01,^DJI,NaT,^DJI,NaN,24824.01,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-29,26064.02,None,None
1,2018-06-01,^GSPC,NaT,^GSPC,NaN,2695.81,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-29,2897.52,None,None
2,2018-06-01,^IXIC,NaT,^IXIC,NaN,6511.34,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-29,8030.04,None,None
3,2018-06-01,AAPL,NaT,EA,5000.0,174.48,NaN,NaN,2018-07-31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-29,219.70,None,None
4,2018-08-17,AAPL,2018-09-21,OC,50.0,1.82,225.0,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-29,219.70,217.88,None
5,2018-06-01,AGN,NaT,EA,4000.0,163.99,187.5,NaN,2018-07-26,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-29,187.72,None,None
6,2018-08-17,AGN,2018-09-21,PC,40.0,187.50,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-29,187.72,None,None
7,2018-06-28,AMAT,NaT,EL,NaN,45.29,NaN,NaN,2018-08-16,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-29,43.40,None,None
8,2018-06-01,BABA,NaT,EA,3000.0,186.00,NaN,NaN,2018-08-16,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-29,178.19,None,None
9,2018-08-17,BABA,2018-09-21,OC,40.0,1.34,187.5,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-29,178.19,176.85,None


In [105]:
table.columns

Index([u'TRX Date ', u'SYMBOL', u'Expir Date', u'TRX  CODE', u'Quantity',
       u'BASIS COST\n', u'Strike  Price', u'Call  Premium', u'Market at TRX',
       u'STOCKS-BUY', u'STOCKS-SELL', u'CALLS-BUY', u'CALLS-SELL',
       u'PUTTES-BUY', u'PUTTES-SELL', u'CASH', u'DIVDNT', u'Gain/ (Loss)',
       u'BALANCE'],
      dtype='object')

In [21]:
table_1 = pd.merge(left=table, right=shares_1, left_on='SYMBOL', right_on='SYMBOL')

In [44]:
type(table['TRX Date '][2])

pandas._libs.tslib.Timestamp

In [47]:
a = table['TRX Date '][2]

In [64]:
b

'12 22 2017'

In [62]:
b = a.strftime('%m %d %Y')

In [63]:
print pd.to_datetime(b,format='%m %d %Y')

2017-12-22 00:00:00


In [34]:
new_col_names = {'STOCKS':'STOCKS-BUY','Unnamed: 10':'STOCKS-SELL',
                'CALLS':'CALLS-BUY','Unnamed: 12': 'CALLS-SELL',
                'PUTTES':'PUTTES-BUY','Unnamed: 14': 'PUTTES-SELL'}

In [38]:
table_1.rename(columns=new_col_names, inplace=True)

In [39]:
table_1.to_excel('result_' + file_name, index=False)

In [ ]:
shares_panel = web.get_data_yahoo(list(symbl), start, end)

In [13]:
f = web.DataReader('F', 'morningstar', start, end)

NotImplementedError: data_source='morningstar' is not implemented

In [66]:
f

Close   High    Low   Open    Volume
Symbol Date                                            
F      2018-05-09  11.06  11.28  11.02  11.21  43731958
       2018-05-10  11.21  11.25  11.02  11.04  31519940

In [187]:
shares

,symbl,Adj Close,Close,High,Low,Open,Current Date
0,ABT,61.720001,61.720001,64.160004,60.740002,61.000000,2018-01-24
1,BX,36.080002,36.080002,36.330002,35.680000,36.250000,2018-01-24
2,CMCSA,42.990002,42.990002,44.000000,41.570000,42.700001,2018-01-24
3,CVX,131.389999,131.389999,132.669998,131.070007,131.380005,2018-01-24
4,MDT,85.760002,85.760002,86.129997,85.160004,85.830002,2018-01-24
5,MS,57.240002,57.240002,57.730000,56.900002,57.209999,2018-01-24
6,PM,110.599998,110.599998,110.889999,109.559998,109.559998,2018-01-24
7,SUN,32.540001,32.540001,33.040001,32.419998,32.529999,2018-01-24
8,UN,58.240002,58.240002,58.320000,57.990002,58.279999,2018-01-24


In [14]:
symbl

array([u'CMCSA', u'SUN', u'CVX', u'MDT', u'ABT', u'BX', u'PM', u'UN', u'MS'], dtype=object)

In [163]:
panel = web.DataReader(symbl, 'yahoo', start_date, end_date)

In [114]:
aaa.index.levels[0][0]

Timestamp('2018-01-24 00:00:00')

In [177]:
aaa.index.names

FrozenList([u'Date', u'minor'])

In [188]:
aaa.drop(start_date - timedelta(days=1) , level=0, axis=0)

Adj Close       Close        High         Low        Open  \
Date       minor                                                               
2018-01-24 ABT     61.720001   61.720001   64.160004   60.740002   61.000000   
           BX      36.080002   36.080002   36.330002   35.680000   36.250000   
           CMCSA   42.990002   42.990002   44.000000   41.570000   42.700001   
           CVX    131.389999  131.389999  132.669998  131.070007  131.380005   
           MDT     85.760002   85.760002   86.129997   85.160004   85.830002   
           MS      57.240002   57.240002   57.730000   56.900002   57.209999   
           PM     110.599998  110.599998  110.889999  109.559998  109.559998   
           SUN     32.540001   32.540001   33.040001   32.419998   32.529999   
           UN      58.240002   58.240002   58.320000   57.990002   58.279999   

                      Volume  
Date       minor              
2018-01-24 ABT    11704900.0  
           BX      5629000.0  
           CMCSA  48918800.0  
           CVX     6868200.0  
           MDT     5680200.0  
           MS      8403600.0  
           PM      4404100.0  
           SUN      490200.0  
           UN      1281200.0

In [149]:
from datetime import timedelta

In [151]:
start_date - timedelta(days=1)

datetime.date(2018, 1, 23)

In [173]:
start_date - timedelta(days=1)

datetime.date(2018, 1, 23)

In [4]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-05-09,186.550003,187.399994,185.220001,187.360001,186.640305,23211200


In [56]:
data.columns

AttributeError: 'Panel' object has no attribute 'columns'